In [1]:
# This code is based on example code from https://github.com/rorodata/background-removal

import os
import sys
from PIL import Image
from scipy.misc import imresize
import numpy as np
from keras.models import load_model
import tensorflow as tf
import io
import requests
import base64
from keras.preprocessing import image as KerasImage
%matplotlib inline
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [2]:
MODEL_URL = 'https://gitlab.com/fast-science/background-removal-server/raw/master/webapp/model/main_model.hdf5'
MODEL_PATH = '/notebooks/work/main_model.hdf5'

def download_model():
    """Downloads the model file.
    """
    if os.path.exists(MODEL_PATH):
        print("Model file is already downloaded.")
        return
    # Download to a tmp file and move it to final file to avoid inconsistent state
    # if download fails or cancelled.
    print("Model file is not available. downloading...")
    exit_status = os.system("wget {} -O {}.tmp".format(MODEL_URL, MODEL_PATH))
    if exit_status == 0:
        os.system("mv {}.tmp {}".format(MODEL_PATH, MODEL_PATH))
    else:
        print("Failed to download the model file" )
        sys.exit(1)

# Preload our model
download_model()
print("Loading model")
model = load_model(MODEL_PATH, compile=False)
graph = tf.get_default_graph()

Model file is already downloaded.
Loading model


/usr/local/lib/python2.7/dist-packages/keras/engine/topology.py:1271: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [3]:
def ml_predict(image):
    with graph.as_default():
        # Add a dimension for the batch
        prediction = model.predict(image[None, :, :, :])
    prediction = prediction.reshape((224,224, -1))
    return prediction


# THRESHOLD = 0.5
def predict_image(image, THRESHOLD=0.5):
    image_as_array = KerasImage.img_to_array(image)
    """Removed the background of given image.
    :param image: numpy array
    """
    height, width = image_as_array.shape[0], image_as_array.shape[1]
    resized_image = imresize(image_as_array, (224, 224)) / 255.0

    # Model input shape = (224,224,3)
    # [0:3] - Take only the first 3 RGB channels and drop ALPHA 4th channel in case this is a PNG
    prediction = ml_predict(resized_image[:, :, 0:3])
    print('PREDICTION COUNT', (prediction[:, :, 1]>THRESHOLD).sum())

    # Resize back to original image size
    # [:, :, 1] = Take predicted class 1 - currently in our model = Person class. Class 0 = Background
    prediction = imresize(prediction[:, :, 1], (height, width))
    
    prediction[prediction<THRESHOLD*255] = 0
    prediction[prediction>=THRESHOLD*255] = 1
    
    #return prediction

    res1=prediction*image_as_array[0:,:,0]
    res2=prediction*image_as_array[0:,:,1]
    res3=prediction*image_as_array[0:,:,2]
    img2=np.dstack([res1,res2,res3])
    return KerasImage.array_to_img(img2)

In [4]:

files_in_input_dir = ! cd input && ls

In [5]:
print(len(files_in_input_dir))

36


In [6]:
def is_image_file(item):
    if item.endswith('.jpg') or item.endswith('.png'):
        return True
    else: 
        return False


image_filenames = filter(is_image_file, files_in_input_dir)
# image_filenames

for filename in image_filenames:
    print(filename)
    input_path = 'input/'+filename
    image = KerasImage.load_img(input_path)
    result = predict_image(image)
    print('results done')
    output_path = 'output/'+'.'.join(filename.split('.')[:-1])+'.png'
#     result_resized = result
    
    # just want to hardwire these couple 
    result_resized = result.resize( (112,137) )
#   or make it half the original
#     result_resized = result.resize( (result.size[0]/2, result.size[1]/2 ) )
    result_resized.save(output_path, transparency=[0,0,0])

P00003186.jpg


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  app.launch_new_instance()


('PREDICTION COUNT', 27544)
results done
P00003251.jpg


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


('PREDICTION COUNT', 30002)
results done
P00003392.jpg
('PREDICTION COUNT', 27095)
results done
P00003608.jpg
('PREDICTION COUNT', 28758)
results done
P20002671.jpg
('PREDICTION COUNT', 30181)
results done
P20002804.jpg
('PREDICTION COUNT', 37788)
results done
P20003984.jpg
('PREDICTION COUNT', 32266)
results done
P40002347.jpg
('PREDICTION COUNT', 30533)
results done
P40003576.jpg
('PREDICTION COUNT', 28703)
results done
P60003076.jpg
('PREDICTION COUNT', 22156)
results done
P60003654.jpg
('PREDICTION COUNT', 32059)
results done
P60003670.jpg
('PREDICTION COUNT', 35211)
results done
P60006111.jpg
('PREDICTION COUNT', 24803)
results done
P60006731.jpg
('PREDICTION COUNT', 4615)
results done
P60007168.jpg
('PREDICTION COUNT', 30470)
results done
P60007242.jpg
('PREDICTION COUNT', 18566)
results done
P60009636.jpg
('PREDICTION COUNT', 30605)
results done
P60009685.jpg
('PREDICTION COUNT', 36569)
results done
P60015948.jpg
('PREDICTION COUNT', 21360)
results done
P60017563.jpg
('PREDICTIO

In [7]:
t = (112,137)
t

(112, 137)

In [8]:
# old
input_path = 'input/'+image_filenames[0]
image = KerasImage.load_img(path)

result = predict_image(image)


output_path = 'output/'+'.'.join(image_filenames[0].split('.')[:-1])+'.png'
result_resized = result.resize( (result.size[0]/2, result.size[1]/2 ) )
result_resized.save(output_path, transparency=[0,0,0])

NameError: name 'path' is not defined